In [1]:
import pandas as pd
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import re
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import IsolationForest

In [154]:
import numpy as np
from tqdm import tqdm

In [3]:
# Function to load JSON data from a file
def load_json_log(file_path):
    data = []
    with open(file_path, 'r') as file:
        count = 0
        for line in file:
            if line.strip():
                line = line.rstrip('\n')
                try:
                    data.append(json.loads(line))
                    count += 1
                except:
                    print(count)
                    print(line)
                    break
                
    return data

In [4]:
# Function to extract session details
def process_session_info(session_str):
    session_info = {}
    
    # Extract Session ID
    session_id_match = re.search(r'id: ([^\s,]+)', session_str)
    if session_id_match:
        session_info['session_id'] = session_id_match.group(1)
    
    # Extract Local Address and Port
    local_match = re.search(r'L:/([^\s]+):(\d+)', session_str)
    if local_match:
        session_info['local_ip'] = local_match.group(1)
        session_info['local_port'] = int(local_match.group(2))
    
    # Extract Remote Address and Port
    remote_match = re.search(r'R:/([^\s]+):(\d+)', session_str)
    if remote_match:
        session_info['remote_ip'] = remote_match.group(1)
        session_info['remote_port'] = int(remote_match.group(2))
    
    return session_info

In [5]:
# Function to extract session details
def process_queryType(query_type):
    
    query = {'isUnknown':0,
             'isDDL':0,
            'isDQL':0,
            'isDML':0,
            'isDCL':0,
            'isTCL':0,
            }
    isUnknown = True
    
    if pd.isna(query_type):
        query['isUnknown'] = 1
        return query
    
    # DDL Queries
    ddl_keywords = ['create', 'drop', 'alter', 'tablespace', 'type', 'cast', 'index', 'schema', 'function', 'role', 'group']
    
    # DQL Queries (mainly SELECT)
    dql_keywords = ['select']
    
    # DML Queries
    dml_keywords = ['insert', 'update', 'delete']
    
    # DCL Queries
    dcl_keywords = ['policy', 'grant', 'revoke']
    
    # TCL Queries (transaction control)
    tcl_keywords = ['begin', 'commit', 'rollback', 'savepoint', 'transaction']
    
    # Convert queryType to lowercase and check for keywords
    query_type_lower = query_type.lower()
    
    if any(keyword in query_type_lower for keyword in ddl_keywords):
        query['isDDL'] = 1
        isUnknown = False
    
    if any(keyword in query_type_lower for keyword in dql_keywords):
        query['isDQL'] = 1
        isUnknown = False

    if any(keyword in query_type_lower for keyword in dml_keywords):
        query['isDML'] = 1
        isUnknown = False
    
    if any(keyword in query_type_lower for keyword in dcl_keywords):
        query['isDCL'] = 1
        isUnknown = False
    
    if any(keyword in query_type_lower for keyword in tcl_keywords):
        query['isTCL'] = 1
        isUnknown = False


    if  isUnknown:
        query['isUnknown'] = 1
        
    return query

In [6]:
def table_info_extraction(tbl):
    tbl_info={'numTblAccessed':0}
    if pd.isna(tbl):
        return tbl_info

    if tbl[0] != '[':
        tbl_info['numTblAccessed']=1
    else:
        tbls = tbl[1:-1].split(' ')
        tbl_info['numTblAccessed'] = len(tbls)
    return tbl_info

In [155]:
# Main Preprocessing Function
def preprocess_transaction_log(data):

    # Load data into a Pandas DataFrame
    df = pd.json_normalize(data)

    ### 1. Handling NaN Values and drop not-useful columns
    df = df.drop(['transactionId', 'txnStartTime', 'txnEndTime' ,'messages', 'parserError', 'clientQueryBypass'], axis=1)
    df = df.dropna(subset=['dateTime', 'clientQuery']) # drop rows containing nan

    # df = df[0:100]
    
    df['rowCount'].fillna(0, inplace=True)
    df['columnCount'].fillna(0, inplace=True)
    df['dbError'].fillna('None', inplace=True)

    ### 1. DateTime Processing ###
    # Convert 'dateTime' to a proper datetime format
    df['dateTime'] = pd.to_datetime(df['dateTime'])

    # Extract useful features from 'dateTime'
    # df['year'] = df['dateTime'].dt.year # year may not be very useful for anomaly detection
    df['month'] = df['dateTime'].dt.month
    df['day'] = df['dateTime'].dt.day
    df['day_of_week'] = df['dateTime'].dt.dayofweek  # Monday=0, Sunday=6
    df['hour'] = df['dateTime'].dt.hour
    df['minute'] = df['dateTime'].dt.minute
    df['second'] = df['dateTime'].dt.second
    
    # Binary feature indicating if the transaction happened during the weekend
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
    
    # Calculate time difference (in seconds) between consecutive transactions (shift the 'dateTime' to compare)
    # May not be useful information for anomaly detection
    # df['time_since_last_query'] = df['dateTime'].diff().dt.total_seconds().fillna(0)
    # drop dateTime 
    df.drop(['dateTime'], axis=1, inplace=True)


    ### 2. Session Information Extraction ###
    df_expanded = df['session'].apply(process_session_info).apply(pd.Series)
    df = pd.concat([df, df_expanded], axis=1)
    df.drop(['session'], axis=1, inplace=True)

    ### 2. queryType Extraction ###
    df_queryType_expanded = df['queryType'].apply(process_queryType).apply(pd.Series)
    df = pd.concat([df, df_queryType_expanded], axis=1)

    ### 2. table Extraction ###
    df_tbl_expanded = df['table'].apply(table_info_extraction).apply(pd.Series)
    df = pd.concat([df, df_tbl_expanded], axis=1)

    df['isTransformed'] = df['transformedQuery'].notna().astype(int)

    df['isAccessedEncCol'] = df['column'].notna().astype(int)
    
    # Label Encoding for categorical features
    label_cols = ['user', 'client', 'database', 'connection', 'schema', 'session_id', 'local_ip', 'remote_ip']
    label_encoders = {}
    df.reset_index(drop=True, inplace=True)
    print("Encoding Categorical features")
    for col in tqdm(label_cols):
        df[col] = df[col].fillna("None")
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col])
        label_encoders[col] = le  # Store encoders for future use if needed


    # Generate BERT embeddings for each query in 'clientQuery'
    client_query_embeddings = []
    print("Encoding clientQuery")
    for query in tqdm(df['clientQuery']):
        embedding = get_bert_embeddings(query)
        client_query_embeddings.append(embedding.flatten())  # Flatten the embedding to 1D array
    
    # Create a DataFrame for BERT embeddings
    bert_embeddings_df_query = pd.DataFrame(client_query_embeddings, columns=[f'bert_dim_{i}' for i in range(client_query_embeddings[0].shape[0])])

    # Generate BERT embeddings for dbError
    print("Encoding dbError")
    dbError_embeddings = []
    for error in tqdm(df['dbError']):
        embedding = get_bert_embeddings(error)
        dbError_embeddings.append(embedding.flatten())  # Flatten the embedding to 1D array
    
    # Create a DataFrame for BERT embeddings
    bert_embeddings_df_dbError = pd.DataFrame(dbError_embeddings, columns=[f'dbError_bert_dim_{i}' for i in range(dbError_embeddings[0].shape[0])])
    
    # List of numerical features
    numerical_features = ['txnTime', 'queryProcessTime', 'resultProcessTime', 'parserTime', 
                          'rowCount', 'columnCount', 'messageSize', 'local_port', 
                          'remote_port', 'month', 'day', 'day_of_week', 'hour', 'minute', 
                          'second', 'is_weekend', 'isUnknown', 'isDDL', 'isDQL', 'isDML', 
                          'isDCL', 'isTCL', 'numTblAccessed', 'isTransformed', 'isAccessedEncCol']
    
    print("numerical_features fill nan with 0")
    for col in tqdm(numerical_features):
        df[col].fillna(0, inplace=True)

    
    # Combine all feature columns for the model
    features = [col + '_encoded' for col in label_cols] + numerical_features

    combined_df = pd.concat([df[features], bert_embeddings_df_query, bert_embeddings_df_dbError], axis=1)

    ### Step 4: Standardize the combined feature set
    scaler = StandardScaler()
    X = scaler.fit_transform(combined_df)

    return df, X 

In [150]:
# Function to get BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    outputs = bert_model(**inputs)
    # Get the embeddings for the [CLS] token (first token)
    cls_embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return cls_embeddings


In [151]:
# Example Usage: Load and preprocess the transaction log
file_path = 'txns.log'
json_data = load_json_log(file_path)

In [156]:
dff, X = preprocess_transaction_log(json_data)

Encoding Categorical features


100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 48.91it/s]


Encoding clientQuery


100%|█████████████████████████████████████| 36046/36046 [33:43<00:00, 17.81it/s]


Encoding dbError


100%|█████████████████████████████████████| 36046/36046 [17:34<00:00, 34.17it/s]


numerical_features fill nan with 0


100%|█████████████████████████████████████████| 25/25 [00:00<00:00, 1180.47it/s]


In [157]:
with open('embedings.npy', 'wb') as f:
    np.save(f, X)

In [158]:
# Initialize and train the Isolation Forest model
isolation_forest = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)
isolation_forest.fit(X)

IsolationForest(contamination=0.1, random_state=42)

In [161]:
# save the model


In [160]:
# Predict anomalies (-1 is an anomaly, 1 is normal)
df['anomaly'] = isolation_forest.predict(X)

ValueError: Length of values (36046) does not match length of index (36281)

In [144]:
dff[combined_df['anomaly'] == -1]

,user,client,clientQuery,database,txnTime,connection,queryProcessTime,resultProcessTime,parserTime,queryType,...,isTransformed,isAccessedEncCol,user_encoded,client_encoded,database_encoded,connection_encoded,schema_encoded,session_id_encoded,local_ip_encoded,remote_ip_encoded
2,None,/0:0:0:0:0:0:0:1%0,"ALTER DATABASE ""regression"" SET lc_messages TO...",regression,5488,UNSECURED,36,742,33,[sstpostgresqlalterdatabase sstpostgresqlalter...,...,0,0,0,0,1,0,0,2,0,0
4,postgres,/0:0:0:0:0:0:0:1%0,GRANT ALL ON SCHEMA public TO postgres;,regression,3,UNSECURED,0,0,0,NaN,...,0,0,1,0,1,0,0,4,0,0
5,postgres,/0:0:0:0:0:0:0:1%0,GRANT ALL ON SCHEMA public TO public;,regression,2,UNSECURED,0,0,0,NaN,...,0,0,1,0,1,0,0,4,0,0
14,postgres,/0:0:0:0:0:0:0:1%0,CREATE TABLESPACE regress_tblspacewith LOCATIO...,regression,43,UNSECURED,1,13,0,sstpostgresqlCreateTablespace,...,0,0,1,0,1,0,0,3,0,0
20,postgres,/0:0:0:0:0:0:0:1%0,ALTER TABLESPACE regress_tblspace SET (some_no...,regression,3,UNSECURED,0,1,0,NaN,...,0,0,1,0,1,0,0,3,0,0
21,postgres,/0:0:0:0:0:0:0:1%0,ALTER TABLESPACE regress_tblspace RESET (rando...,regression,4,UNSECURED,0,1,0,NaN,...,0,0,1,0,1,0,0,3,0,0
23,postgres,/0:0:0:0:0:0:0:1%0,CREATE SCHEMA testschema;,regression,8,UNSECURED,1,0,2,sstcreateschema,...,0,0,1,0,1,0,0,3,0,0
53,postgres,/0:0:0:0:0:0:0:1%0,CREATE TABLE testschema.part_1 PARTITION OF te...,regression,5,UNSECURED,0,1,2,sstcreatetable,...,0,0,1,0,1,0,0,3,0,0
59,postgres,/0:0:0:0:0:0:0:1%0,CREATE TABLE testschema.part_3 PARTITION OF te...,regression,8,UNSECURED,0,1,1,sstcreatetable,...,0,0,1,0,1,0,0,3,0,0
65,postgres,/0:0:0:0:0:0:0:1%0,CREATE TABLE testschema.part_78 PARTITION OF t...,regression,5,UNSECURED,1,0,1,sstcreatetable,...,0,0,1,0,1,0,0,3,0,0


In [175]:
cols  = ['user', 'client', 'clientQuery', 'database', 'txnTime', 'connection',
        'queryProcessTime', 'resultProcessTime', 'parserTime', 'dbError',
        'rowCount', 'columnCount', 'messageSize', 'schema', 'month', 'day',
        'day_of_week', 'hour', 'minute', 'second', 'is_weekend', 'session_id',
        'local_ip', 'local_port', 'remote_ip', 'remote_port', 'isUnknown',
        'isDDL', 'isDQL', 'isDML', 'isDCL', 'isTCL', 'numTblAccessed',
        'isTransformed', 'isAccessedEncCol']    

In [181]:
cat_col = ['user', 'client','database', 'connection', 'schema', 'day_of_week', 'is_weekend']

2

In [10]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]